<a href="https://colab.research.google.com/github/MGunnesslal/CV/blob/main/U_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras

In [ ]:
!pip install tifffile

In [ ]:
!pip install pillow

In [ ]:
!pip install rasterio

In [ ]:
!pip install tensorflow-addons

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
import tensorflow_addons as tfa
import rasterio
from rasterio.plot import reshape_as_raster, reshape_as_image
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout
from keras.models import Model
from keras.optimizers import Adam

In [8]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
#Define constants
IMAGE_FOLDER = '/content/drive/MyDrive/mband/imgTif'
MASK_FOLDER = '/content/drive/MyDrive/mband/mask'
IMAGE_SIZE = 128
BANDS = 8
NUM_CLASSES = 8
BATCH_SIZE = 32
EPOCHS = 10

In [10]:
# Define color mappings for classes
class_colors = {
    (125, 235, 247): 0,  # Water
    (109, 49, 9): 1,     # Coconut
    (34, 99, 19): 2,     # Forest
    (227, 238, 88): 3,   # Agriculture
    (132, 252, 104): 4,  # Mangrove
    (20, 188, 132): 5,   # Non-Forested Wetland
    (252, 166, 180): 6,  # Sandy Beach
    (255, 0, 0): 7       # Urban Development
}

In [23]:
# Data loading and preprocessing
def load_data(image_folder, mask_folder):
    images = []
    masks = []
    for filename in os.listdir(image_folder):
        if filename.endswith('.tif'):
            image_path = os.path.join(image_folder, filename)
            mask_path = os.path.join(mask_folder, filename[:-4] + '.png')
            images.append(image_path)
            masks.append(mask_path)
    return images, masks

def preprocess_image(img):
    # Normalize image to range [0, 1]
    img = img / 65535.0
    return img

def preprocess_mask(mask):
    # Convert RGBA mask to class indices
    mask_rgb = cv2.cvtColor(mask, cv2.COLOR_RGBA2RGB)
    mask_indices = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.uint8)
    for color, idx in class_colors.items():
        indices = np.all(mask_rgb == np.array(color), axis=-1)
        mask_indices[indices] = idx
    return mask_indices

def preprocess_data(image_paths, mask_paths):
    images = []
    masks = []
    for img_path, mask_path in zip(image_paths, mask_paths):
        with rasterio.open(img_path) as src:
            img = src.read()
            img = np.transpose(img, axes=(1, 2, 0))  # Transpose to have channels last
        img = preprocess_image(img)
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        mask = preprocess_mask(mask)
        images.append(img)
        masks.append(mask)
    return np.array(images), np.array(masks)

In [14]:
# Split data into training and validation sets
def split_data(images, masks):
    train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)
    return train_images, val_images, train_masks, val_masks

In [15]:
# Data augmentation
def data_augmentation(image, mask):
    # Perform data augmentation here
    # For example, you can use rotation, flipping, zooming, etc.
    return image, mask

In [16]:
# U-Net model
def unet_model(nb_classes=NUM_CLASSES, img_size=IMAGE_SIZE, nb_channels=BANDS, nb_filters_start=32,
               growth_factor=2, upconv=True, class_weights=[0.3, 0.5, 0.2, 0.2, 0.4, 0.3, 0.4, 0.4],
               droprate=0.25):
    nb_filters = nb_filters_start
    inputs = Input(shape=(img_size, img_size, nb_channels))  # Adjust input shape dynamically
    conv1 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    nb_filters *= growth_factor
    pool1 = BatchNormalization()(pool1)
    conv2 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(droprate)(pool2)

    nb_filters *= growth_factor
    pool2 = BatchNormalization()(pool2)
    conv3 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(droprate)(pool3)

    nb_filters *= growth_factor
    pool3 = BatchNormalization()(pool3)
    conv4_0 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(pool3)
    conv4_0 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(conv4_0)
    pool4_1 = MaxPooling2D(pool_size=(2, 2))(conv4_0)
    pool4_1 = Dropout(droprate)(pool4_1)

    nb_filters *= growth_factor
    pool4_1 = BatchNormalization()(pool4_1)
    conv4_1 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(pool4_1)
    conv4_1 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(conv4_1)
    pool4_2 = MaxPooling2D(pool_size=(2, 2))(conv4_1)
    pool4_2 = Dropout(droprate)(pool4_2)

    nb_filters *= growth_factor
    conv5 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(pool4_2)
    conv5 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(conv5)


    # this is the bottom part of "U"


    nb_filters //= growth_factor
    if upconv:
        up6_1 = concatenate([Conv2DTranspose(nb_filters, (2, 2), strides=(2, 2), padding='same')(conv5), conv4_1])
    else:
        up6_1 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4_1])

    up6_1 = BatchNormalization()(up6_1)
    conv6_1 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(up6_1)
    conv6_1 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(conv6_1)
    conv6_1 = Dropout(droprate)(conv6_1)

    nb_filters //= growth_factor
    if upconv:
        up6_2 = concatenate([Conv2DTranspose(nb_filters, (2, 2), strides=(2, 2), padding='same')(conv6_1), conv4_0])
    else:
        up6_2 = concatenate([UpSampling2D(size=(2, 2))(conv6_1), conv4_0])
    up6_2 = BatchNormalization()(up6_2)
    conv6_2 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(up6_2)
    conv6_2 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(conv6_2)
    conv6_2 = Dropout(droprate)(conv6_2)

    nb_filters //= growth_factor
    if upconv:
        up7 = concatenate([Conv2DTranspose(nb_filters, (2, 2), strides=(2, 2), padding='same')(conv6_2), conv3])
    else:
        up7 = concatenate([UpSampling2D(size=(2, 2))(conv6_2), conv3])
    up7 = BatchNormalization()(up7)
    conv7 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = Dropout(droprate)(conv7)

    nb_filters //= growth_factor
    if upconv:
        up8 = concatenate([Conv2DTranspose(nb_filters, (2, 2), strides=(2, 2), padding='same')(conv7), conv2])
    else:
        up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2])
    up8 = BatchNormalization()(up8)
    conv8 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = Dropout(droprate)(conv8)

    nb_filters //= growth_factor
    if upconv:
        up9 = concatenate([Conv2DTranspose(nb_filters, (2, 2), strides=(2, 2), padding='same')(conv8), conv1])
    else:
        up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1])
    conv9 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(nb_filters, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(nb_classes, (1, 1), activation='softmax')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    # Define loss function
    def weighted_binary_crossentropy(y_true, y_pred):
        class_loglosses = tf.reduce_mean(tf.keras.backend.binary_crossentropy(y_true, y_pred), axis=[0, 1, 2])
        return tf.reduce_sum(class_loglosses * tf.constant(class_weights))

    model.compile(optimizer=Adam(), loss=weighted_binary_crossentropy)

    return model

In [17]:
# Load data
image_paths, mask_paths = load_data(IMAGE_FOLDER, MASK_FOLDER)
images, masks = preprocess_data(image_paths, mask_paths)

In [18]:
# Split data
train_images, val_images, train_masks, val_masks = split_data(images, masks)

In [19]:
# Data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_masks))
train_dataset = train_dataset.map(data_augmentation)
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_masks))
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [20]:
# Build and compile the model
input_shape = (IMAGE_SIZE, IMAGE_SIZE, BANDS)
model = unet_model()
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
# Train the model
model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset)

Epoch 1/20
1/1 [==============================] - 21s 21s/step - loss: 2.0430 - accuracy: 0.2307 - val_loss: 2.0688 - val_accuracy: 1.0000
Epoch 2/20
1/1 [==============================] - 7s 7s/step - loss: 1.8292 - accuracy: 0.8046 - val_loss: 2.0503 - val_accuracy: 1.0000
Epoch 3/20
1/1 [==============================] - 9s 9s/step - loss: 1.4034 - accuracy: 0.9852 - val_loss: 2.0143 - val_accuracy: 1.0000
Epoch 4/20
1/1 [==============================] - 8s 8s/step - loss: 0.8064 - accuracy: 0.9983 - val_loss: 1.9222 - val_accuracy: 1.0000
Epoch 5/20
1/1 [==============================] - 8s 8s/step - loss: 0.2591 - accuracy: 0.9996 - val_loss: 1.4902 - val_accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 10s 10s/step - loss: 0.0435 - accuracy: 1.0000 - val_loss: 0.4419 - val_accuracy: 1.0000
Epoch 7/20
1/1 [==============================] - 9s 9s/step - loss: 0.0096 - accuracy: 1.0000 - val_loss: 0.0645 - val_accuracy: 1.0000
Epoch 8/20
1/1 [=====================

In [22]:
# Evaluate the model on the validation dataset
val_loss, val_accuracy = model.evaluate(val_dataset)

print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

1/1 [==============================] - 1s 1s/step - loss: 6.8460e-05 - accuracy: 1.0000
Validation Loss: 6.84604310663417e-05
Validation Accuracy: 1.0
